In [1]:
import nltk
from nltk.corpus import wordnet
import pandas as pd
import numpy as np
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
data = pd.read_csv("mazda_newest_gm_reviews.csv")

In [3]:
data = data.loc[:,['caption','rating']]

In [4]:
data.head()

,caption,rating
0,We had a wonderful experience at pine belt Maz...,5
1,NaN,5
2,Ryan Santos is professional and honest in hand...,5
3,I had my old car in for major service and I ha...,5
4,Great experience with service and sales!,5


In [5]:
data.isnull().sum()

caption    26
rating      0
dtype: int64

In [6]:
data['caption'].fillna("None",inplace=True)

In [6]:
def negation_handler(sentence):
    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't","'t"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    for i in range(len(sentence)):
      if "ca" == sentence[i]:
        sentence[i]="can"
    return sentence

In [9]:
sentence =nltk.word_tokenize("This is not pretty good pencil")
sen = " ".join(negation_handler(sentence))
sen

'This is unreasonably good pencil'

In [8]:
def caption_modification(sentence):
    sentence =nltk.word_tokenize(sentence)
    sentence = negation_handler(sentence)
    return " ".join(sentence)

In [9]:
data['caption'] = data['caption'].apply(caption_modification)

In [10]:
def magicword_remover(sentence):
    sentence = "".join(char for char in sentence if char.isalnum() or char.isspace() or char==".")
    return " ".join(sentence.split())
# print(magicword_remover("code for removing ! !    unnecessary space from string in between words").strip())

In [11]:
data['caption'] = data['caption'].apply(magicword_remover)

In [12]:
data['caption'].iloc[103]

'I could differ happier with my experience at Pine Belt Mazda I came to them after my car was totalled and had to start from nothing . The team worked with me and helped get me in a new car less than a week after my accident . They made it'

In [13]:
nltk.download('vader_lexicon')
def sentiment_analysis(text):
    sentiment = SentimentIntensityAnalyzer()
    score = sentiment.polarity_scores(text)
    if score['compound'] > 0.5:
        return 'Positive'
    elif score['compound'] < -0.5:
        return 'Negative'
    else:
        return 'Neutral'

text = "I could differ happier with my experience at Pine Belt Mazda I came to them after my car was totalled and had to start from nothing . The team worked with me and helped get me in a new car less than a week after my accident . They made it"
print("this ",sentiment_analysis(text))

this  Neutral


[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [14]:
data['polarityVader'] = data['caption'].apply(sentiment_analysis)

In [15]:
data['polarityVader'].iloc[155]

'Positive'

In [16]:
data['caption'].iloc[155]

'From someone who dislikes every aspect of car shopping this place was great in my opinion . Our salesman was Seamus who was very personable and genuine . He was efficient and helpful throughout the whole process . There are a bunch of car'

In [17]:
data['caption'].iloc[103]

'I could differ happier with my experience at Pine Belt Mazda I came to them after my car was totalled and had to start from nothing . The team worked with me and helped get me in a new car less than a week after my accident . They made it'

In [18]:
data['polarityVader'].iloc[103]

'Neutral'

In [19]:
def get_polarity(comment):
    polarity = TextBlob(comment).sentiment.polarity
    subjectivity = TextBlob(comment).sentiment.subjectivity
    if polarity>0:
        return "positive"
    elif polarity ==0:
        return "neutral"
    else:
        return "negative"

In [20]:
data["Polaritytxtblob"] = data["caption"].apply(get_polarity)

In [21]:
data.iloc[165]

caption            The worest dealership I have ever deal with . ...
rating                                                             1
polarityVader                                               Positive
Polaritytxtblob                                             negative
Name: 165, dtype: object

In [22]:
get_polarity("I could differ happier with my experience at Pine Belt Mazda. I came to them after my car was totalled and had to start from nothing. The team worked with me and helped get me in a new car less than a week after my accident. They made it")

'negative'

In [23]:
data.head()

,caption,rating,polarityVader,Polaritytxtblob
0,We had a wonderful experience at pine belt Maz...,5,Positive,positive
1,None,5,Neutral,neutral
2,Ryan Santos is professional and honest in hand...,5,Positive,positive
3,I had my old car in for major service and I ha...,5,Positive,positive
4,Great experience with service and sales,5,Positive,positive


In [29]:
data["polarityVader"].value_counts()

Positive    179
Neutral      59
Negative      2
Name: polarityVader, dtype: int64

In [31]:
data["Polaritytxtblob"].value_counts()

positive    197
neutral      34
negative      9
Name: Polaritytxtblob, dtype: int64

In [32]:
data["rating"].value_counts()

5    217
1     14
3      4
4      3
2      2
Name: rating, dtype: int64

In [27]:
# import torch
# from transformers import pipeline, set_seed

# set_seed(42)

sentiment_model = pipeline("sentiment-analysis")

def analyze_sentiment(text):
    result = sentiment_model(text)
    sentiment = result[0]['label']
    score = result[0]['score']
    
    if sentiment == 'LABEL_NEGATIVE':
        sentiment = 'Negative'
    elif sentiment == 'LABEL_POSITIVE':
        sentiment = 'Positive'
    else:
        sentiment = 'Neutral'
    
    return sentiment, score

text = "I love this product! It's amazing."
print(analyze_sentiment(text)) # Output: ('Positive', 0.999)

text = "This product is not good. I don't like it."
print(analyze_sentiment(text)) # Output: ('Negative', 0.994)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


ValueError: Could not load model distilbert-base-uncased-finetuned-sst-2-english with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>).